In [1]:
import os

In [2]:
pwd

'/home/dephinate/ASU/TextSummarization/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/home/dephinate/ASU/TextSummarization'

Create Entity 

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



Add Methods in Configuration manager to read Configurations from config.yaml for Data Ingestion and create directories to support Data Ingestion

In [7]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            param_filepath=PARAMS_FILE_PATH) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])



    # Method to read Data Ingestion Config

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


Update the Components to download data

In [9]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.utils.common import get_size
from TextSummarizer.logging import logger

In [15]:
class DataIngestion:
    def __init__(
            self,
            config: DataIngestionConfig
            ) -> None:
        self.config = config

    # Method to download the file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
                )
            logger.info(f'{filename} download! with the following info: \n{header}')
        else:
            logger.info(f"File already exits of size: {get_size(Path(self.config.local_data_file))}")

    # Method to Extract the downloaded file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

Update the Pipeline

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-01-10 18:04:58,064,INFO,common,yaml file config/config.yaml loaded successfully]
[2024-01-10 18:04:58,065,INFO,common,yaml file params.yaml loaded successfully]
[2024-01-10 18:04:58,066,INFO,common,created directory at: artifacts]
[2024-01-10 18:04:58,067,INFO,common,created directory at: artifacts/data_ingestion]
[2024-01-10 18:04:58,773,INFO,2427399911,artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BFD4:7D21:20563E:295684:659F3E73
Accept-Ranges: bytes
Date: Thu, 11 Jan 2024 01:04:58 GMT
Via: 1.1 varnish
X-Served-By: cache-lax-kwhp1940034-LAX
X-Cache: HIT
X-Cac